In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit import Parameter
import numpy as np
from qiskit.circuit.library import XGate
from qiskit.visualization import array_to_latex, plot_bloch_multivector, visualize_transition, plot_histogram
from qiskit.quantum_info import Statevector
from qiskit.circuit import Parameter
from qiskit_aer.primitives import Sampler
from qiskit.result import marginal_counts

[IBM Quantum Platform](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwiwm8H8gcyBAxUOIUQIHezkACgQjBB6BAgMEAE&url=https%3A%2F%2Fquantum-computing.ibm.com%2Flogin&usg=AOvVaw1GwPlk_pqjt5PQJQ59hlEB&opi=89978449)

[2023 Qiskit Global Summer School](https://www.youtube.com/playlist?list=PLOFEBzvs-VvqoeIypXYLLf0PY-WOQMLR3)

### Lab 3 Review
---
- Q1: What are the two basis states of a Qubit? What is the default basis state of a Qubit?

- Q2: Are Standard Gates reversible?

- Q3: What is a XGate? -- What is it's axis of rotation? How much does it rotate by?
- Q4: What is a YGate? -- What is it's axis of rotation? How much does it rotate by?
- Q5: What is a ZGate? -- What is it's axis of rotation? How much does it rotate by?

- Q6: If you needed to rotate with respect to the X-axis some arbitrary amount, which standard gate would you use?
- Q7: If you needed to rotate with respect to the Y-axis some arbitrary amount, which standard gate would you use?
- Q8: If you needed to rotate with respect to the Z-axis some arbitrary amount, which standard gate would you use?

- Q9: What are the purpose of Control Gates? The CXGate operates on how many Qubits?

- Q10: Is the measure standard operation reversible? 

In [ ]:
ket0 = [[1], [0]]
array_to_latex(ket0)

Qiskit's [`Statevector`](https://qiskit.org/documentation/stubs/qiskit.quantum_info.Statevector.html) class can take different forms of input (e.g. python list, numpy array, another state vector) to construct a state vector.

Let's take the `ket0` object we created earlier and convert it to a `Statevector` object:

In [ ]:
state_0 = Statevector(ket0)
plot_bloch_multivector(state_0)

In [ ]:
ket1 = [[0], [1]]
array_to_latex(ket1)

In [ ]:
state_1 = Statevector(ket1)
plot_bloch_multivector(state_1)

In [ ]:
x = XGate()
x_matrix = x.to_matrix()
array_to_latex(x_matrix)

In [ ]:
matrix_ex = x_matrix @ ket0
state_0_X = Statevector(matrix_ex)
plot_bloch_multivector(state_0_X, title="X|0>")

In [ ]:
array_to_latex(matrix_ex)

#### Problem #1
---
Given the visualized transition below, apply a standard gate to return the state to |0>. There is multiple correct ways to do this. However, an optimized solution (minimal circuit depth) only requires one gate.

In [ ]:
qc = QuantumCircuit(1)
qc.x(0)
qc.h(0)
qc.y(0)
#YOUR CODE GOES HERE
qc.ry(-np.pi/2,0)

visualize_transition(qc, trace=True, fpg=30)

#### Problem #2
--- 
Given the visualized transition below, apply a single gate to put the qubit in the |1> state. Similiar to the problem above, there is multiple correct solutions.

In [ ]:
qc = QuantumCircuit(1)
qc.h(0)
qc.h(0)
#YOUR CODE GOES HERE
qc.x(0)
visualize_transition(qc, fpg=30, trace=True)

#### Problem #3
---
Given the visualized transition below, apply a single gate to set the qubit's state to |1>

In [ ]:
qc = QuantumCircuit(1)
qc.x(0)
qc.h(0)
#YOUR CODE GOES HERE
qc.ry(-np.pi/2,0)
visualize_transition(qc, fpg=30, trace=True)

#### Problem #4
---
Unlike the previous problems, you do not have to figure out which gate to use. Apply an RXGate to qubit 0, to set the state of the qubit to it's default state.

In [ ]:
qc = QuantumCircuit(1)
qc.x(0)
qc.ry(np.pi/2, 0)
qc.rz(-np.pi/2, 0)
#YOUR CODE GOES HERE
qc.rx(np.pi/2,0)
visualize_transition(qc, trace=True, fpg=30)

- Due top the No Cloning Theorem, Quantum Information cannot be copied. Quantum Telportation provides the ability for the original state to be reconstructed. This is inherently distructive for the original qubit. The reason for the Quantum States distruction is the Standard Operation, measurement. 


In [ ]:
q = QuantumCircuit(QuantumRegister(1,"Control"))
phase_angle = Parameter('T')
q.ry(phase_angle, 0)

q.draw(output="mpl")

## Quantum Teleportation using a Bell State

In [ ]:
#Copy the initial circuit
qc = q.copy()

#Attach registers necessary for teleportation
bell = QuantumRegister(2, 'Bell')
alice = ClassicalRegister(2, 'Alice')
bob = ClassicalRegister(1, 'Bob')

qc.add_register(bell, alice, bob)

qc.barrier()

#Create Bell state (Entagle qubits bell[0] and bell[1])
qc.h(bell[0])
qc.cx(bell[0],bell[1])

qc.barrier()

#Alice applies operations to her qubit's
qc.cx(0,1)
qc.h(0)

qc.barrier()

#Alice measures her qubits, to send the results to Bob
qc.measure(0,alice[0])
qc.measure(bell[0],alice[1])

# This is an example of dynamic circuits
# We can create a circuit that only applies an operation if a certain state is measured
with qc.if_test((alice[1],1)):
    qc.x(bell[1])
    with qc.if_test((alice[0],1)):
        qc.z(bell[1])

qc.barrier()

#finally we can measure Bob's qubit to recover the state
qc.measure(bell[1], bob)
qc.draw(output="mpl")

In [ ]:
#The initial circuit with only an angle parameter
q.measure_all()

In [ ]:
#The angle we want to test
angle = 2*np.pi/7

#Sample is necessary because we are using Dynamic Circuits which do not work with simulator
sampler = Sampler()

#Run the circuit with the angle parameter
before_tele = sampler.run(q.assign_parameters({phase_angle: angle}))
before = before_tele.result().quasi_dists[0].binary_probabilities()
plot_histogram(before)

In [ ]:
# NOTE this block is used to visualize the operations made by Alice
temp_circ = QuantumCircuit(1)
temp_circ.ry(angle, 0)
temp_circ.h(0)

visualize_transition(temp_circ, trace=True, fpg=30)

In [ ]:
#Run the teleportation circuit with binded parameter 
after_tele = sampler.run(qc.bind_parameters({phase_angle: angle}))


after = after_tele.result().quasi_dists[0].binary_probabilities()

print("Original probabilities: ",before)
print("Teleported probabilities: ",after)


Notice that binary probabilities are different between the circuits? The original circuit only contains a single classical register while the teleportation circuit contains three. However, Bob is only interested in his Classical Register `Bob`(Third ClassicalRegister, index 2). Qiskit's [marginal_counts](https://qiskit.org/documentation/apidoc/result.html#qiskit.result.marginal_counts) method provides a mean of combining probabilities of a certain index.

In [ ]:
teleported_counts = marginal_counts(after, indices=[2])

In [ ]:
legend = ['Before Teleportation', 'After Teleportation']
plot_histogram([before_tele.result().quasi_dists[0].binary_probabilities(),teleported_counts],legend=legend)